#### In this notebook, we will create a database in our MS SQL Server and populate it with synthetic data.

In [5]:
import pyodbc
import sys
import random
import pandas as pd
from faker import Faker


#### We've already created a database called `testDB1` on the MS SQL Server containing 3 empty tables: 

`CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(100),
    ContactName VARCHAR(100),
    Country VARCHAR(50)
);`

`CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(100),
    SupplierID INT,
    CategoryID INT,
    UnitPrice DECIMAL(10, 2)
);`

`CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATETIME,
    TotalAmount DECIMAL(10, 2),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);`

In [6]:
# establish connection with the local SQL server
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=172.16.6.196,1433;"  # Use the IP address and port directly
    "Database=testDB1;"  
    "UID=wsl;" 
    "PWD=greatpond501;"  
)

print(conn_str)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# test the connection
cursor.execute("SELECT @@version;")

Driver={ODBC Driver 17 for SQL Server};Server=172.16.6.196,1433;Database=testDB1;UID=wsl;PWD=greatpond501;


#### Now let's insert some fake data into our tables, we will use `pyodbc` to directly run SQL queries on the database. 

In [7]:
# faker object for generating random fake data
fake = Faker()

In [22]:
num_rows = [1000, 2000, 5000]

customer_ids = random.sample(range(1, num_rows[0] + 1), num_rows[0])
product_ids = random.sample(range(1, num_rows[1] + 1), num_rows[1])
order_ids = random.sample(range(1, num_rows[2] + 1), num_rows[2])


# Clear out the tables
cursor.execute("DELETE FROM Orders")
cursor.execute("DELETE FROM Products")
cursor.execute("DELETE FROM Customers")
conn.commit()

In [23]:
# insert data into Customers table
for customer_id in customer_ids:
    cursor.execute("INSERT INTO Customers (CustomerID, CustomerName, ContactName, Country) VALUES (?, ?, ?, ?)",
                   (customer_id, fake.company(), fake.name(), fake.country()))

# insert data into Products table
for product_id in product_ids:
    cursor.execute("INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, UnitPrice) VALUES (?, ?, ?, ?, ?)",
                   (product_id, fake.word(),fake.random_int(min=1, max=100), fake.random_int(min=1, max=100), fake.random_number(digits=2)))

# insert data into Orders table
for order_id in order_ids:
    cursor.execute("INSERT INTO Orders (OrderID, CustomerID, OrderDate, TotalAmount) VALUES (?, ?, ?, ?)",
                   (order_id, random.choice(customer_ids), fake.date_time_this_decade(), fake.random_number(digits=2)))


cursor.commit()
conn.close()   


Now we're done populating the tables with synthetic data. Let's do a sanity check on the row counts.

In [25]:
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

queries = {"Customers" : "SELECT Count(*) FROM Customers", 
           "Products" : "SELECT Count(*) FROM Products",
           "Orders" : "SELECT Count(*) FROM Orders"}

# execute the queries and get the results
for table, query in queries.items():
    cursor.execute(query)
    rows = cursor.fetchone()[0]
    print(f"{table} table --> {rows} rows")


# show first 10 rows from each table
for table in queries.keys():
    query = f"SELECT TOP 10 * FROM {table}"
    df = pd.read_sql(query, conn)
    print(f"\n{table} table")
    print(df)

conn.close()    

Customers table --> 1000 rows
Products table --> 2000 rows
Orders table --> 5000 rows

Customers table
   CustomerID                    CustomerName         ContactName  \
0           1     Norman, Preston and Vasquez   Elizabeth Cordova   
1           2                 Noble-Rodriguez       Kristin Smith   
2           3                   Howard-Medina       Kevin Holland   
3           4                     Cole-Rogers      Clarence Davis   
4           5  Lopez, Crawford and Livingston      Angela Roberts   
5           6                      Clay-Davis        William Rose   
6           7                   Miller-Greene     Laura Patterson   
7           8      Ochoa, Lawrence and Conner  Christopher Brooks   
8           9                    Gonzalez LLC    Patricia Watkins   
9          10                    Terry-Jensen        Maxwell Khan   

                  Country  
0                  Tuvalu  
1              Mozambique  
2                 Denmark  
3  Libyan Arab Jamahiriya

/tmp/ipykernel_90071/172073778.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/tmp/ipykernel_90071/172073778.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/tmp/ipykernel_90071/172073778.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
